In [1]:
import os
import json
import random
import itertools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model

# Define dataset paths
base_dir = "/kaggle/input/the-wildfire-dataset/the_wildfire_dataset_2n_version"
train_dir = f"{base_dir}/train"
val_dir = f"{base_dir}/val"
test_dir = f"{base_dir}/test"

IMG_SIZE = (128, 128)
BATCH_SIZE = 16
SEED = 1234

# Load datasets correctly
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode="binary", seed=SEED
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode="binary", seed=SEED
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode="binary", seed=SEED
)

# Define normalization layer
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

# Apply normalization before batching
def preprocess_ds(dataset):
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))
    return dataset.prefetch(tf.data.AUTOTUNE)  # Avoids bottlenecks

# Apply preprocessing
train_ds = preprocess_ds(train_ds)
val_ds = preprocess_ds(val_ds)
test_ds = preprocess_ds(test_ds)

# Define hyperparameter search space for NASNetMobile
HYPERPARAMS = {
    "lr": [0.0001, 0.001, 0.01],  # Learning rate
    "bs": [16, 32, 64],  # Batch size
    "drop": [0.2, 0.3, 0.5],  # Dropout rate
    "dense": [128, 256, 512],  # Dense layer size
    "aug": [True, False],  # Data augmentation on/off
    "base_trainable": [True, False],  # Fine-tune NASNetMobile or freeze it
    "stem_block_filters": [32, 64, 96],  # Number of filters in the initial stem block
    "activation": ["relu", "swish"],  # Activation function for dense layers
}

EPOCHS = 10  
INPUT_SHAPE = (128, 128, 3)

# Generate all possible hyperparameter combinations
all_combinations = list(itertools.product(*HYPERPARAMS.values()))
N_RANDOM_TRIALS = min(10, len(all_combinations))  
random_combinations = random.sample(all_combinations, N_RANDOM_TRIALS)

# Function to create a NASNetMobile model
def create_cnn_model(lr, drop, dense, aug, base_trainable, stem_block_filters, activation):
    """Creates and compiles a NASNetMobile-based model with flexible hyperparameters."""
    base_model = tf.keras.applications.NASNetMobile(
        input_shape=(128, 128, 3), include_top=False, weights="imagenet"
    )
    base_model.trainable = base_trainable  # Fine-tune or freeze the base model

    inputs = keras.Input(shape=(128, 128, 3))
    
    # Apply augmentation before passing through NASNetMobile
    x = aug(inputs) if aug else inputs

    x = base_model(x, training=base_trainable)  # Fine-tuning setting
    x = layers.GlobalAveragePooling2D()(x)  # Converts feature maps into a single vector
    x = layers.Dense(dense, activation=activation)(x)
    x = layers.Dropout(drop)(x)  # Dropout for regularization
    outputs = layers.Dense(1, activation="sigmoid")(x)  # Binary classification

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Directory to save models
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

best_val_acc = 0
best_model_info = {}
results = []  # Store results for all hyperparameter combinations

# Training loop
for i, hyperparams in enumerate(random_combinations, start=1):
    print(f"\nTraining {i}/{N_RANDOM_TRIALS}: {hyperparams}...")

    # Unpack hyperparameters
    lr, bs, drop, dense, aug, base_trainable, stem_block_filters, activation = hyperparams

    # Define data augmentation if enabled
    data_augmentation = (
        keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
            layers.Resizing(128, 128)  # Ensures augmentation does not change image size
        ])
        if aug else None
    )

    # Create the NASNetMobile model
    model = create_cnn_model(lr, drop, dense, data_augmentation, base_trainable, stem_block_filters, activation)

    # Apply augmentation correctly
    if aug:
        train_ds_aug = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    else:
        train_ds_aug = train_ds

    # **EARLY STOPPING CALLBACK**
    early_stopping = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2, restore_best_weights=True)

    # Train the model with no logs (verbose=0)
    history = model.fit(
        train_ds_aug,  
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[early_stopping],
        verbose=0  # Hide training logs
    )

    print(f"  Early stopping triggered at epoch {len(history.history['accuracy'])}.")

    # Evaluate on test dataset
    test_loss, test_acc = model.evaluate(test_ds, verbose=0)

    # Store results (kept the same format as before)
    model_info = {
        "Combination": i,
        "Learning Rate": lr,
        "Batch Size": bs,
        "Dropout": drop,
        "Dense": dense,
        "Augmentation": aug,
        "Base Model Trainable": base_trainable,
        "Stem Block Filters": stem_block_filters,
        "Activation": activation,
        "Train Accuracy": history.history["accuracy"][-1],
        "Val Accuracy": history.history["val_accuracy"][-1],
        "Test Accuracy": test_acc,
    }
    results.append(model_info)

    # Track the best model
    if model_info["Val Accuracy"] > best_val_acc:
        best_val_acc = model_info["Val Accuracy"]
        best_model_info = model_info

        # Save the best model (same format as before)
        best_model_path = os.path.join(save_dir, "best_model.keras")
        model.save(best_model_path)

        # Save training history (same format as before)
        best_history_path = os.path.join(save_dir, "best_history.json")
        with open(best_history_path, "w") as f:
            json.dump(history.history, f)

        print(f"  New best model saved: {best_model_path}")

# Save all results (same format as before)
results_path = os.path.join(save_dir, "all_results.json")
with open(results_path, "w") as f:
    json.dump(results, f)

# Save best model info (same format as before)
best_info_path = os.path.join(save_dir, "best_model_info.json")
with open(best_info_path, "w") as f:
    json.dump(best_model_info, f)

print(f"\nAll results saved to: {results_path}")
print(f"Best model details saved to: {best_info_path}")
print(f"Best model file: {best_model_path}")
print(f"Best training history file: {best_history_path}")

Found 1887 files belonging to 2 classes.
Found 402 files belonging to 2 classes.
Found 410 files belonging to 2 classes.

Training 1/10: (0.001, 64, 0.5, 512, True, True, 96, 'swish')...
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
  Early stopping triggered at epoch 8.
  New best model saved: saved_models/best_model.keras

Training 2/10: (0.001, 16, 0.2, 256, True, False, 96, 'relu')...
  Early stopping triggered at epoch 10.
  New best model saved: saved_models/best_model.keras

Training 3/10: (0.01, 16, 0.2, 128, False, True, 32, 'swish')...
  Early stopping triggered at epoch 3.

Training 4/10: (0.01, 64, 0.2, 128, False, True, 96, 'swish')...
  Early stopping triggered at epoch 3.

Training 5/10: (0.0001, 64, 0.3, 128, True, True, 96, 'relu')...
  Early stopping triggered at epoch 8.

Training 6/10: (0.001, 64, 0.2, 256, False, False, 64, 'relu')...
  Early stopping triggered at epoch 3.
  New best model saved: saved_models/best_model.keras

Training 7/10: (0.0001, 16, 0.2, 